In [1]:
import json
from py2neo import Graph, Node

In [2]:
graph = Graph("http://localhost:7474", auth=("neo4j", "test"))
if graph:
    print("connect successfully")

connect successfully


In [3]:
# read data

# 7 nodes
drugs = []
foods = []
checks = []
departments = []
producers = []
diseases = []
symptoms = []
# disease infos
disease_infos = []
# 11 relationships
rels_department = []
rels_noteat = []
rels_doeat = []
rels_recommandeat = []
rels_commonddrug = []
rels_recommanddrug = []
rels_check = []
rels_drug_producer = []
rels_symptom = []
rels_acompany = []
rels_category = []

data_path = "data/medical2.json"
for data in open(data_path):
    disease_dict = {}
    data_json = json.loads(data)
    
    disease_dict['name'] = disease = data_json['name']
    diseases.append(disease)
    
    disease_dict['desc'] = ''
    disease_dict['prevent'] = ''
    disease_dict['cause'] = ''
    disease_dict['easy_get'] = ''
    disease_dict['cure_department'] = ''
    disease_dict['cure_way'] = ''
    disease_dict['cure_lasttime'] = ''
    disease_dict['symptom'] = ''
    disease_dict['cured_prob'] = ''
    
    if 'symptom' in data_json:
        # add symptom list
        symptoms += data_json['symptom']
        # add symptom relationships
        for symptom in data_json['symptom']:
            rels_symptom.append([disease, symptom])
    
    if 'acompany' in data_json:
        for acompany in data_json['acompany']:
            rels_acompany.append([disease, acompany])
    
    if 'desc' in data_json:
        disease_dict['desc'] = data_json['desc']
    
    if 'prevent' in data_json:
        disease_dict['prevent'] = data_json['prevent']
    
    if 'cause' in data_json:
        disease_dict['cause'] = data_json['cause']
    
    if 'get_prob' in data_json:
        disease_dict['get_prob'] = data_json['get_prob']
    
    if 'easy_get' in data_json:
        disease_dict['easy_get'] = data_json['easy_get']
    
    if 'cure_department' in data_json:
        cure_department = data_json['cure_department']
        if len(cure_department) == 1:
            rels_category.append([disease, cure_department[0]])
        elif len(cure_department) == 2:
            big = cure_department[0]
            small = cure_department[1]
            rels_department.append([small, big])
            rels_category.append([disease, small])
    
    if 'cure_way' in data_json:
        disease_dict['cure_way'] = data_json['cure_way']
        
    if 'cure_lasttime' in data_json:
        disease_dict['cure_lasttime'] = data_json['cure_lasttime']
    
    if 'cured_prob' in data_json:
        disease_dict['cured_prob'] = data_json['cured_prob']
    
    if 'common_drug' in data_json:
        common_drug = data_json['common_drug']
        for drug in common_drug:
            rels_commonddrug.append([disease, drug])
        drugs += common_drug
        
    if 'recommand_drug' in data_json:
        recommand_drug = data_json['recommand_drug']
        drugs += recommand_drug
        for drug in recommand_drug:
            rels_recommanddrug.append([disease, drug])
    
    if 'not_eat' in data_json:
        not_eat = data_json['not_eat']
        for _not in not_eat:
            rels_noteat.append([disease, _not])
        
        foods += not_eat
        do_eat = data_json['do_eat']
        for _do in do_eat:
            rels_doeat.append([disease, _do])
        
        foods += do_eat
        recommand_eat = data_json['recommand_eat']
        
        for _recommand in recommand_eat:
            rels_recommandeat.append([disease, _recommand])
        foods += recommand_eat
    
    if 'check' in data_json:
        check = data_json['check']
        for _check in check:
            rels_check.append([disease, _check])
        checks += check
    
    if 'drug_detail' in data_json:
        drug_detail = data_json['drug_detail']
        producer = [i.split('(')[0] for i in drug_detail]
        rels_drug_producer += [[i.split('(')[0], 
                                i.split('(')[-1].replace(')', '')] 
                               for i in drug_detail]
        producers += producer
    
    disease_infos.append(disease_dict)

Drugs = set(drugs)
Foods = set(foods)
Checks = set(checks)
Departments = set(departments)
Producers = set(producers)
Symptoms = set(symptoms)
diseases = set(diseases)

In [4]:
# create disease nodes
for disease_dict in disease_infos:
    node = Node("Disease",
                name=disease_dict['name'],
                desc=disease_dict['desc'],
                prevent=disease_dict['prevent'],
                cause=disease_dict['cause'],
                easy_get=disease_dict['easy_get'],
                cure_lasttime=disease_dict['cure_lasttime'],
                cure_department=disease_dict['cure_department'],
                cure_way=disease_dict['cure_way'],
                cured_prob=disease_dict['cured_prob'],
               )
    graph.create(node)

In [5]:
# create drug nodes
for drug in Drugs:
    node = Node("Drug", name=drug)
    graph.create(node)

In [6]:
# create food nodes
for food in Foods:
    node = Node("Food", name=food)
    graph.create(node)

In [7]:
# create check nodes
for check in Checks:
    node = Node("Check", name=check)
    graph.create(node)

In [8]:
# create department nodes
for department in Departments:
    node = Node("Department", name=department)
    graph.create(node)

In [9]:
# create producer nodes
for t in Producers:
    node = Node("Producer", name=t)
    graph.create(node)

In [10]:
# create symptom nodes
for t in Symptoms:
    node = Node("Symptom", name=t)
    graph.create(node)

In [11]:
# create relationship
def create_relationship(start_node, end_node, edges, rel_type, rel_name):
    # deduplicate
    set_edges = []
    for edge in edges:
        set_edges.append('###'.join(edge))
    set_edges = set(set_edges)
    num_edges = len(set_edges)
    for edge in set_edges:
        edge = edge.split('###')
        p = edge[0]
        q = edge[1]
        query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (
                start_node, end_node, p, q, rel_type, rel_name)
        try:
            graph.run(query)
        except Exception as e:
            print(e)

In [12]:
create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')